## Chapter 5: Resampling Methods Lab

In [4]:
import numpy as np
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize, poly)
from sklearn.model_selection import train_test_split
from functools import partial
from sklearn.model_selection import (cross_validate, KFold, ShuffleSplit)
from sklearn.base import clone
from ISLP.models import sklearn_sm

### The Validation Set Approach

In [5]:
auto = load_data('Auto')
auto_train, auto_valid = train_test_split(auto, 
                                          test_size=196, 
                                          random_state=0)

In [7]:
# Fit linear regression
hp_mm = MS(['horsepower'])
x_train = hp_mm.fit_transform(auto_train)
y_train = auto_train['mpg']
model = sm.OLS(y_train, x_train)
results = model.fit()

In [8]:
# Predict on validation set
x_valid = hp_mm.transform(auto_valid)
y_valid = auto_valid['mpg']
valid_pred = results.predict(x_valid)
np.mean((y_valid - valid_pred)**2) # Validation MSE

23.61661706966988

In [9]:
# Function to return MSE on test set
def evalMSE(terms,
            response,
            train,
            test):
    mm = MS(terms)
    X_train = mm.fit_transform(train)
    y_train = train[response]
    X_test = mm.transform(test)
    y_test = test[response]
    results = sm.OLS(y_train, X_train).fit()
    test_pred = results.predict(X_test)
    return np.mean((y_test - test_pred)**2)

In [11]:
MSE = np.zeros(3)

# Evaluate linear, quadratic, and cubic fits
for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       auto_train,
                       auto_valid)
MSE

array([23.61661707, 18.76303135, 18.79694163])

In [16]:
# Randomize train-test split
auto_train, auto_valid = train_test_split(auto, 
                                          test_size=196, 
                                          random_state=3)
MSE = np.zeros(3)
for idx, degree in enumerate(range(1,4)):
    MSE[idx] = evalMSE([poly('horsepower', degree)],
                       'mpg',
                       auto_train,
                       auto_valid)
MSE

array([20.75540796, 16.94510676, 16.97437833])

### Cross-Validation

In [17]:
hp_model = sklearn_sm(sm.OLS, MS(['horsepower']))
x,y = auto.drop(columns=['mpg']), auto['mpg']

cv_results = cross_validate(hp_model, x, y, cv=auto.shape[0])
cv_err = np.mean(cv_results['test_score'])
cv_err

24.23151351792924

In [18]:
# Cross-validation for polynomials of degrees 1-5
cv_err = np.zeros(5)
h = np.array(auto['horsepower'])
m = sklearn_sm(sm.OLS)

for i, d in enumerate(range(1,6)):
    x = np.power.outer(h, np.arange(d+1))
    m_cv = cross_validate(m, x, y, cv= auto.shape[0])
    cv_err[i] = np.mean(m_cv['test_score'])
cv_err

array([24.23151352, 19.24821312, 19.33498406, 19.42443033, 19.03323827])

In [21]:
# K-Fold Cross-Validation
cv_err = np.zeros(5)
cv = KFold(n_splits=10, shuffle=True, random_state=0)

for i,d in enumerate(range(1,6)):
    x = np.power.outer(h, np.arange(d+1))
    m_cv = cross_validate(m, x, y, cv=cv)
    cv_err[i] = np.mean(m_cv['test_score'])
cv_err

array([24.20766449, 19.18533142, 19.27626666, 19.47848403, 19.13720581])

In [23]:
# Implement using ShuffleSplit()
validation = ShuffleSplit(n_splits=10, test_size=196, random_state=0)
results = cross_validate(hp_model, auto.drop(['mpg'], axis=1),
                         auto['mpg'], cv=validation)
results['test_score'].mean(), results['test_score'].std()

(23.802232661034168, 1.421845094109185)

### The Bootstrap

In [24]:
portfolio = load_data('Portfolio')

# Compute estimate for alpha
def alpha_func(D, idx):
    cov_ = np.cov(D[['X', 'Y']].loc[idx], rowvar=False)
    return((cov_[1,1] - cov_[0,1]) / (cov_[0,0] + cov_[1,1] - 2*cov_[0,1]))

In [26]:
alpha_func(portfolio, range(100))

0.57583207459283

In [27]:
# New bootstrap data set
rng = np.random.default_rng(0)
alpha_func(portfolio, rng.choice(100, 100, replace=True))

0.6074452469619004

In [28]:
# Compute bootstrap standard error 
def boot_SE(func, D, n=None, B=1000, seed=0):
    rng = np.random.default_rng(seed)
    first_, second_ = 0, 0
    n = n or D.shape[0]
    for _ in range(B):
        idx = rng.choice(D.index, n, replace=True)
        value = func(D, idx)
        first_ += value
        second_ += value**2
    return np.sqrt(second_ / B - (first_ / B)**2)

In [29]:
# Evaluate alpha estimate
alpha_SE = boot_SE(alpha_func, portfolio, B=1000, seed=0)
alpha_SE

0.09118176521277699

In [42]:
# Bootstrap a regression model 
def boot_OLS(model_matrix, response, D, idx):
    D_ = D.loc[idx]
    Y_ = D_[response]
    X_ = clone(model_matrix).fit_transform(D_)
    return sm.OLS(Y_, X_).fit().params

In [45]:
hp_func = partial(boot_OLS, MS(['horsepower']), 'mpg')

In [46]:
hp_se = boot_SE(hp_func, auto, B=1000, seed=10)
hp_se

intercept     0.731176
horsepower    0.006092
dtype: float64

In [47]:
hp_model.fit(auto, auto['mpg'])
model_se = summarize(hp_model.results_)['std err']
model_se

intercept     0.717
horsepower    0.006
Name: std err, dtype: float64

In [48]:
# Quadratic model
quad_model = MS([poly('horsepower', 2, raw=True)])
quad_func = partial(boot_OLS, quad_model, 'mpg')
boot_SE(quad_func, auto, B=1000)

intercept                                  1.538641
poly(horsepower, degree=2, raw=True)[0]    0.024696
poly(horsepower, degree=2, raw=True)[1]    0.000090
dtype: float64

In [49]:
M = sm.OLS(auto['mpg'], quad_model.fit_transform(auto))
summarize(M.fit())['std err']

intercept                                  1.800
poly(horsepower, degree=2, raw=True)[0]    0.031
poly(horsepower, degree=2, raw=True)[1]    0.000
Name: std err, dtype: float64